In [1]:
import pandas as pd
import numpy as np

raw_files = ['../datasets/raw/CDD CSV Export - LC123 Primary.csv',
            '../datasets/raw/CDD CSV Export - LC123 Retest.csv',
            '../datasets/raw/CDD CSV Export - LC4 Primary.csv',
            '../datasets/raw/CDD CSV Export - MLPCN Primary.csv',
            '../datasets/raw/CDD CSV Export - MLPCN Retest.csv']

for r in raw_files:
    df = pd.read_csv(r)
    print(df['Library ID'].unique())

['LC-3' 'Life Chemicals 1' nan 'Life Chemical 2']
['LC-3' 'Life Chemicals 1' nan '2011 MLPCN' '2011MLPCN' 'NCC1&amp;2'
 'Life Chemical 2']
['LC-4' nan]
['2011 MLPCN' nan '2011MLPCN']
['2011 MLPCN' '2011MLPCN' 'Life Chemicals 1' 'Life Chemical 2']


In [18]:
df = pd.read_csv('../datasets/master_df/master_mlpcn_lc_2018_10_18.csv.gz')

In [2]:
for r in raw_files:
    df = pd.read_csv(r)
    print(df.columns)

Index(['Molecule Name', 'SMILES', 'Batch Name', 'Library ID', 'Plate Name',
       'Plate Well',
       'PriA-SSB AS, normalized for plate and edge effects, correct plate map: Run Date',
       'PriA-SSB AS, normalized for plate and edge effects, correct plate map: % inhibition Alpha, normalized (%)'],
      dtype='object')
Index(['Molecule Name', 'SMILES', 'Batch Name', 'Library ID', 'Plate Name',
       'Plate Well',
       'PriA-SSB AS, normalized for plate and edge effects, correct plate map: Run Date',
       'PriA-SSB AS, normalized for plate and edge effects, correct plate map: % inhibition Alpha, normalized (%)'],
      dtype='object')
Index(['Molecule Name', 'SMILES', 'Batch Name', 'Library ID', 'Plate Name',
       'Plate Well',
       'PriA-SSB AS, normalized for plate and edge effects, correct plate map: Run Date',
       'PriA-SSB AS, normalized for plate and edge effects, correct plate map: % inhibition Alpha, normalized (%)'],
      dtype='object')
Index(['Molecule Name'

In [24]:
df2 = pd.read_csv('../datasets/raw/merged_cdd_2018_10_15.csv.gz')
df2.columns, df2.shape

(Index(['Unnamed: 0', 'SMSSF ID', 'CDD SMILES', 'Batch Name', 'Library ID',
        'Plate Name', 'Plate Well', 'Run Date', 'PriA-SSB AS % inhibition'],
       dtype='object'), (442274, 9))

In [14]:
import pandas as pd
import numpy as np

folds_df = [pd.read_csv('../datasets/folds/library_strat_folds/fold_{}.csv'.format(i)) for i in range(10)]
train_df = pd.read_csv('../datasets/folds/training_df_single_fold_with_clustering.csv.gz')
train_df['Fold ID'] = 0

for i, tmp_df in enumerate(folds_df):
    merge_df = pd.merge(train_df, tmp_df, on='Molecule ID')
    assert merge_df.shape[0] == tmp_df.shape[0]
    molid_list = merge_df['Molecule ID'].tolist()
    train_df.loc[train_df['Molecule ID'].isin(molid_list),'Fold ID'] = i
    
train_df.to_csv('../datasets/folds/training_df_single_fold_with_clustering_v2.csv.gz', index=False)

In [11]:
import pandas as pd
import numpy as np

cluster_col = 'TB_0.4 ID'
train_df = pd.read_csv('../datasets/folds/training_df_single_fold_with_clustering_v2.csv.gz')

fold_info = []
all_fold_clusters = []
for i in range(10):
    fold_df = train_df[train_df['Fold ID'] == i]
    rem_df = train_df[train_df['Fold ID'] != i]
    
    fold_clusters = np.unique(fold_df[cluster_col].values)
    rem_clusters = np.unique(rem_df[cluster_col].values)
    
    
    n_cpd = fold_df.shape[0]
    n_clusters = fold_clusters.shape[0]
    n_unique = np.setdiff1d(fold_clusters, rem_clusters).shape[0]
    
    all_fold_clusters.append(np.setdiff1d(fold_clusters, rem_clusters))
    
    fold_info.append([i, n_cpd, n_clusters, n_unique])

n_cpd = train_df.shape[0]
n_clusters = train_df[cluster_col].unique().shape[0]

fold_info.append(['All Folds', n_cpd, n_clusters, 0])
fold_info = pd.DataFrame(data=fold_info,
                        columns=['Fold ID', '# Compounds', '# Clusters in Fold', '# Fold Clusters not in other Folds'])
fold_info

,Fold ID,# Compounds,# Clusters in Fold,# Fold Clusters not in other Folds
0,0,42729,26753,3377
1,1,42730,26734,3392
2,2,42737,26696,3276
3,3,42734,26709,3326
4,4,42730,26708,3352
5,5,42729,26772,3342
6,6,42730,26746,3397
7,7,42727,26717,3345
8,8,42727,26783,3442
9,9,42727,26667,3386


In [12]:
print(fold_info.to_latex(index=False))

\begin{tabular}{lrrr}
\toprule
   Fold ID &  \# Compounds &  \# Clusters in Fold &  \# Fold Clusters not in other Folds \\
\midrule
         0 &        42729 &               26753 &                                3377 \\
         1 &        42730 &               26734 &                                3392 \\
         2 &        42737 &               26696 &                                3276 \\
         3 &        42734 &               26709 &                                3326 \\
         4 &        42730 &               26708 &                                3352 \\
         5 &        42729 &               26772 &                                3342 \\
         6 &        42730 &               26746 &                                3397 \\
         7 &        42727 &               26717 &                                3345 \\
         8 &        42727 &               26783 &                                3442 \\
         9 &        42727 &               26667 &                  

In [13]:
from functools import reduce
reduce(np.union1d, all_fold_clusters).shape

(33635,)

In [16]:
100.0 * 33635/88324

38.08138218377791